In [ ]:
import os
import sys
from keras.models import Model
from keras.layers import concatenate
from keras.utils.data_utils import get_file
from keras.optimizers import RMSprop
from data_generator.config import mpii_sp_dataconf
from data_generator.mpii import MpiiSinglePerson
from data_generator.loader import BatchLoader
from models.losses import pose_regression_loss
from models import reception
from models.mpii_tools import eval_singleperson_pckh
from models.mpii_tools import MpiiEvalCallback
from keras.callbacks import LearningRateScheduler
from models.callbacks import SaveModel
from data_generator.io_operate import *
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
num_blocks = 8
batch_size = 12
input_shape = mpii_sp_dataconf.input_shape
num_joints = 16
dataset_path = './data/mpii/'
weights_path = './weights_mpii_001.h5'
TEST_MODE = 0
TRAIN_MODE = 1
VALID_MODE = 2

In [ ]:
weights_path1 = './weights_PE_MPII_cvpr18_19-09-2017.h5'
weights_path2 = './weights_mpii_120.h5'

In [ ]:
model = reception.build(input_shape, num_joints, dim=2,num_blocks=num_blocks, num_context_per_joint=2, ksize=(5, 5))

In [ ]:
model.load_weights(weights_path2)

In [ ]:
mpii = MpiiSinglePerson(dataset_path, dataconf=mpii_sp_dataconf)
mpii_val = BatchLoader(mpii, x_dictkeys=['frame'],y_dictkeys=['pose', 'afmat', 'headsize'], mode=VALID_MODE,\
              batch_size=mpii.get_length(VALID_MODE), num_predictions=1,shuffle=False)

In [ ]:
[x_val], [p_val, afmat_val, head_val] = mpii_val[0]

In [ ]:
inputs = [x_val]
pred = model.predict(inputs, batch_size=batch_size, verbose=1)

In [ ]:
from data_generator.transform import transform_pose_sequence

In [ ]:
y_pred = pred[7]
y_pred = y_pred[:, :, 0:2]
y_pred = transform_pose_sequence(afmat_val.copy(), y_pred, inverse=True)

In [ ]:
y_pred

In [ ]:
import numpy as np 

In [ ]:
save_path = './test/y_pred.npy'

In [ ]:
np.save(save_path,y_pred)

In [ ]:
import numpy as np 
import os
import cv2
import scipy.io as scio 

In [1]:
img_path = './data/mpii/images'
annot_path = './data/mpii/annotation/model_test.mat'
save_path = './test/y_pred.npy'
out_path = './test/test_img'

In [ ]:
y_pred = np.load(save_path)
annot = scio.loadmat(annot_path)
annot = annot['annot_val']

y_pred = y_pred.astype(np.int16)
point_color = [(255,0,0),(0,255,0),(0,0,255),(128,128,0),(0,128,128),(128,0,128)]
map_from_mpii = [6, 7, 8, 9, 12, 13, 11, 14, 10, 15, 2, 3, 1, 4, 0, 5]
map_to_mpii = [14, 12, 10, 11, 13, 15, 0, 1, 2, 3, 8, 6, 4, 5, 7, 9]
y_pred = y_pred[:,map_to_mpii,:]
peo_num = len(annot)
connect = [[0,1,2,6],[6,3,4,5],[6,7,8,9],[8,12,11,10],[8,13,14,15]]

In [ ]:
for i in range(peo_num):
    img_name = annot[i]['name'][0][0][0][:-1]
    img = cv2.imread(os.path.join(out_path,img_name))
    for j in range(len(connect)):
        connecting = connect[j]
        center0 = (y_pred[i][connecting[0]][0],y_pred[i][connecting[0]][1])
        center1 = (y_pred[i][connecting[1]][0],y_pred[i][connecting[1]][1])
        center2 = (y_pred[i][connecting[2]][0],y_pred[i][connecting[2]][1])
        center3 = (y_pred[i][connecting[3]][0],y_pred[i][connecting[3]][1])
        cv2.line(img,center0 ,center1,point_color[j],5)
        cv2.line(img,center1 ,center2,point_color[j],5)
        cv2.line(img,center2 ,center3,point_color[j],5) 
    cv2.imwrite(os.path.join(out_path,img_name), img)

In [ ]:
map_from_mpii = [6, 7, 8, 9, 12, 13, 11, 14, 10, 15, 2, 3, 1, 4, 0, 5]
map_to_mpii = [14, 12, 10, 11, 13, 15, 0, 1, 2, 3, 8, 6, 4, 5, 7, 9]
y_pred = y_pred[:,map_to_mpii,:]
img_name = annot[0]['name'][0][0][0][:-1]
img = cv2.imread(os.path.join(img_path,img_name))
for i in range(16):
    center = (y_pred[0][i][0],y_pred[0][i][1])
    cv2.circle(img, center,5,point_color[0], -1)
    cv2.imwrite(os.path.join(out_path,img_name[:9]+'_'+str(i)+img_name[9:]),img)

In [ ]:
cv2.imshow('input_image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import shutil
img_name = os.listdir('./test/test_img')
del(img_name[7])
for i in img_name:
    srcimg = os.path.join(img_path,i)
    dstimg = os.path.join(out_path,i)
    shutil.copyfile(srcimg,dstimg)

In [25]:
import PIL.Image as Image
import os
img_path = './ntu_pose/mpii_3d_pose'
img_format = ['.jpg', '.JPG']  # 图片格式
size_col = 918  # 每张小图片的大小
size_row = 426
img_row = 2 # 图片间隔，也就是合并成一张图后，一共有几行
img_column = 3  # 图片间隔，也就是合并成一张图后，一共有几列
img_save_path = os.path.join(img_path,'final.jpg')
 
# 获取图片集地址下的所有图片名称
#img_names = os.listdir(img_path)
#del(img_names[7])
#for i in range(11):
    #path = os.path.join(img_path,img_names[i])
    #if(os.path.isfile(path)):
     #   img = Image.open(path)
      #  print(img.size)

In [26]:
img_name = ['14.jpg','17.jpg','18.jpg','4.jpg','5.jpg','7.jpg']

In [27]:
to_image = Image.new('RGB', (img_column * size_col, img_row * size_row)) #创建一个新图
for y in range(img_row):
    for x in range(img_column):
        path = os.path.join(img_path,img_name[3*y + x])
        if(os.path.isfile(path)):
            from_image = Image.open(path)#.resize((img_size, img_size),Image.ANTIALIAS)
            to_image.paste(from_image, (x * size_col, y * size_row))
to_image.save(img_save_path) 


In [15]:
del(img_names[6])

In [16]:
img_names

['000391837.jpg',
 '000377819.jpg',
 '000172482.jpg',
 '000156511.jpg',
 '000025245.jpg',
 '000040154.jpg',
 '000338399.jpg',
 '000142834.jpg']

In [19]:
img = Image.open('./ntu_pose/mpii_3d_pose/17.jpg')

In [28]:
img.size

(918, 426)